In [1]:
pip install openai

In [2]:
pip install langchain

In [3]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
#Import all the necessary libraries

import openai
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

In [5]:
openai.api_key = "************************"

In [19]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=openai.api_key)

/tmp/ipython-input-925182223.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=openai.api_key)


In [20]:
llm.invoke("What is EDA?")

'\n\nEDA stands for Exploratory Data Analysis. It is a process of analyzing and visualizing data to uncover patterns, trends, and relationships that may exist within the data. EDA is often the first step in the data analysis process and helps to identify key variables, outliers, and potential problems with the data. It involves using various statistical and data visualization techniques to gain insights and make informed decisions. EDA is an essential step in data science, machine learning, and business intelligence.'

#Using Custom Dataset

In [6]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 28.5 MB/s eta 0:00:00


In [7]:
pdf_reader = PyPDFLoader("/content/RAGPaper.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
chunks = text_splitter.split_documents(documents)

In [14]:
!pip install tiktoken
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 63.6 MB/s eta 0:00:00


In [15]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

#create embeddings
embeddings = OpenAIEmbeddings(api_key=openai.api_key)
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

In [21]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [22]:
chat_history = []
query = """Does the vendor have experience with similar industries and use cases?​?​​"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

/tmp/ipython-input-1879224906.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"question": query, "chat_history": chat_history})


 The second piece of context provides information about a study that was conducted on a model that can generate questions based on input text. It mentions that the model was evaluated on two criteria: factuality and specificity. This suggests that the model is capable of generating questions relevant to specific industries and use cases. However, it does not explicitly mention whether the model has experience with similar industries and use cases. Therefore, it is not clear whether the vendor has experience with similar industries and use cases.


In [23]:
chat_history = []
query = """?Does the vendor's financial offer make sense compared to the timeline, resources, and deliverables??"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])


I am not sure, as the context provided does not mention anything about a vendor or financial offer.


In [24]:
chat_history = []
query = """?What is RAGs?"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])


RAG (Retrieval-Augmented Generation) is a model used for question answering tasks that combines the flexibility of "closed-book" (parametric only) approaches with the performance of "open-book" retrieval-based approaches. It uses external knowledge sources, such as Wikipedia, to retrieve relevant documents and generate correct answers. RAG has achieved state-of-the-art results in open-domain question answering tasks. 
